In [66]:
# BUSINESS CASE PART
# Task 1
import os
import pandas as pd

# Step 1: Load and merge all files Table_*
path = 'C:/Users/lenovo/Desktop/Uday CBRE Data Analyst Assessment/Python_test'
files = [file for file in os.listdir(path) if file.startswith('Table_') and (file.endswith('.csv') or file.endswith('.xlsx'))]
merged_data = pd.concat([pd.read_csv(os.path.join(path, file)) if file.endswith('.csv') else pd.read_excel(os.path.join(path, file)) for file in files])


# Step 2: Drop duplicates
merged_data.drop_duplicates(inplace=True)
# CSV file unable to read Ź symbol and ? was present, so replacing the ? to Ź again
merged_data['Type'] = merged_data['Type'].replace('\?', 'Ź', regex = True)


# Step 3: Show number of null values in each column
print(merged_data.isnull().sum())


# Step 4: Fill numerical null values with 1337
numeric_columns = merged_data.select_dtypes(include=['number']).columns     # select only numerical data
merged_data[numeric_columns] = merged_data[numeric_columns].fillna(value=1337)


# Step 5: Calculate difference in days between Acctg Date and Date columns
# Convert 'Acctg Date' and 'Date' columns to datetime objects with format given in dataset which is day is 1st
merged_data['Acctg Date'] = pd.to_datetime(merged_data['Acctg Date'], errors='coerce', dayfirst=True)
merged_data['Date'] = pd.to_datetime(merged_data['Date'], errors='coerce', dayfirst=True)

merged_data['Date_Difference'] = (merged_data['Acctg Date'] - merged_data['Date']).dt.days.abs()     # to avoid minus(-) values used .abs()


# Step 6: Calculate difference in BUSINESS days between Acctg Date and Date columns (ignore weekends and UK bank holidays)
from pandas_market_calendars import get_calendar


uk_calendar = get_calendar("XLON")  # Use XLON for UK market calendar
merged_data = merged_data.dropna(subset=['Acctg Date', 'Date'])  # Handle missing values

# Used get_calendar and valid_days to calculate the business days difference for each row for UK
merged_data['Business_Days_Difference'] = merged_data.apply(lambda row: uk_calendar.valid_days(start_date=min(row['Acctg Date'], row['Date']),
                                                                                               end_date=max(row['Acctg Date'], row['Date'])).shape[0],
                                                            axis=1)


# Step 7: Convert Amount to PLN using FXrates.csv
fx_rates = pd.read_csv(path + '/FXrates.csv')
merged_data = pd.merge(merged_data, fx_rates, on='Currency', how='left')
# used formula tgt_amt = src_amt * ( per usd rate of src / per usd rate of tgt)
            #  PLN_amt = src_amt * ( per usd rate of src / per usd rate of PLN)
merged_data['Amount_PLN'] = merged_data['Amount'] * (merged_data['Per USD'] / fx_rates.loc[fx_rates['Currency']=='PLN', 'Per USD'].values[0])
merged_data.drop(['Per USD'],axis = 1, inplace = True)  # droppping Per Usd Column


# Step 8: Save separate files for each unique Type value
output_folder = 'C:/Users/lenovo/Desktop/Uday CBRE Data Analyst Assessment/Results_Pandas_Task-1'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for type_value, type_group in merged_data.groupby('Type'):
    type_group.to_excel(f'{output_folder}/Table_{type_value}.xlsx', index=False)


Voucher          0
Seq              0
Type             0
Account          0
Func Unit     9023
Amount           0
Currency         0
Acctg Date       0
Date            11
Line #           0
Origin           0
dtype: int64
